In [ ]:
from IPython.display import display, HTML, Javascript

In [ ]:
# Create html code to embed the elm generated Javascript code into the notebook...
elm = """<script>
""" + open("elm_counter/elm_counter.js","r").read() + """
</script>
"""

In [ ]:
# Embed the javascript code into the notebook, using the ipython display engine
# This may currently load the Elm script more then once, which will produce an error.
# In that case: clear all outputs, save the notebook and rerun the cells...
display(HTML(elm))

In [ ]:
import traitlets

# The following class uses traitlets to make a copy of the counter state observable
class Counter(traitlets.HasTraits):
    count = traitlets.Int()
    
    # The _repr_html_ method is used to embed the Elm app and provide a DOM node for Elm
    def _repr_html_(self):
        self.__id = 'counter1'
        self.__app = 'app1'
        return """<div id='""" + self.__id + """'></div>
<script>
var """ + self.__app + """ = Elm.Main.init({
    node: document.getElementById('""" + self.__id + """')
});

</script>"""
    
    # Whenever the count value is changed from python, that change is propagated to Elm with javascript
    @traitlets.observe('count')
    def _count_changed(self, change):
        if not change['old'] == change['new']:
            display(Javascript(self.__app + ".ports.portSetValue.send({})".format(self.count)))
    
# Receiving counter changes from Elm requires a subscription on the corresponding port, which has to be done
# in Javascript. (See example below)

In [ ]:
# Create a counter and display the Elm output
c = Counter()
c

In [ ]:
# Set the counter value from python
c.count=4

In [ ]:
# Observe the changes in the Elm output
c.count=6

In [ ]:
# Change the Elm counter using the buttons. This change is not available here, since the callback is not registered.
c.count

In [ ]:
# This class does not use the traitlets, but exposes a property, which calls into the Elm code on write.
# The corresponding callback is registered, however, it relies on the object variables name, which is
# wrong.
# A synchronous call into the Elm code to get the current counter value would be great to implement the
# property getter method.
class Counter2(object):
    def __init__(self):
        self.__id = 'counter2'
        self.__app = 'app2'
    
    # Include a node in the DOM and register a callback for changes of the counter value as well as
    # a request to receive the current value asynchronously for the first time.
    def _repr_html_(self):
        return """<div id='""" + self.__id + """'></div>
<script>
var """ + self.__app + """ = Elm.Main.init({
    node: document.getElementById('""" + self.__id + """')
});

""" + self.__app + """.ports.portSendValue.subscribe(function (value) {
        IPython.notebook.kernel.execute("c2.count="+value);
    });
    
""" + self.__app + """.ports.requestCountValue.send(null);
</script>"""
    
    # Return the shadow copy of the counter value
    @property
    def count(self):
        return self._count
    
    # Update the shadow value of the counter and request Elm to update the counter value in Elm as well
    @count.setter
    def count(self, value):
        self._count = value
        display(Javascript(self.__app + ".ports.portSetValue.send({})".format(value)))

In [ ]:
c2 = Counter2()
c2

In [ ]:
c2.count

In [ ]:
c2.count=6

In [ ]:
c2.count